In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:
torch.version.__version__

'2.2.2'

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [5]:
block_size = 8
batch_size = 4

In [6]:
with open("data/wizard_of_oz.txt", 'r', encoding='utf-8') as file:
    text = file.read()



## Avec un Tokenizer par lettre

In [7]:
chars = sorted(set(text))
print("Liste des caractères présents :")
print(chars, '\n')
vocab_size = len(chars)
print("Nombre de caractères présents :")
print(vocab_size)

Liste des caractères présents :
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff'] 

Nombre de caractères présents :
81


Construction de l'encodage du tokenizer

In [8]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }

encode = lambda string: [string_to_int[char] for char in string]
decode = lambda code  : ''.join([int_to_string[i] for i in code])

In [12]:
## Tests

encoded_hello = encode('hello')
print("'hello' encodé : \n", encoded_hello)
decoded_hello = decode(encoded_hello)
print("\nPuis décodé: \n", decoded_hello)

'hello' encodé : 
 [61, 58, 65, 65, 68]

Puis décodé: 
 hello


### Avec PyTorch

In [15]:
data = torch.tensor(encode(text), dtype=torch.long)

100 premiers caractères encodés

In [16]:
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


#### Split train/test

In [18]:
n = int(0.8 * len(data))
n

185847

In [19]:
train_data = data[:n]
val_data = data[n:]
#Check
len(train_data) + len(val_data)

232309

#### taille de bloc

In [24]:
train_data[88294:88294 + block_size]

tensor([73, 61, 58,  1, 69, 62, 60, 65])

In [25]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y
    
x, y = get_batch('train')
print('inputs :')
print(x)
print('targets :')
print(y)

tensor([133750,   5867, 174518, 145328])
inputs :
tensor([[71, 68, 73, 61, 78, 11,  0,  0],
        [72,  1, 55, 71, 58, 54, 64, 62],
        [62, 67, 64, 65, 58, 57,  1, 59],
        [62, 58, 57,  1, 73, 61, 58, 62]])
targets :
tensor([[68, 73, 61, 78, 11,  0,  0,  3],
        [ 1, 55, 71, 58, 54, 64, 62, 67],
        [67, 64, 65, 58, 57,  1, 59, 58],
        [58, 57,  1, 73, 61, 58, 62, 71]])


In [30]:
class BigramLanguageModel(nn.Module):
    
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss= None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
            
        return index
        

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)

generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

generated_chars

'\n4Kwfgswrj:J1RqM]e MS05BTVxdNa\'oJpQv!BMjIn0\'aqUJZ:P*S2]oHOP-V! j3S,(U8:E;7?01GHc1S&s2g8NE0T,g6LsL,Q]excYpmWs*mdg;)fg6OazT3)A:mXaxmEimEK(wB9ydCME0hh;A2dl-I\n.[m:OXA.RAYAEz7e UhHnh0ah-exWcKK  pMAe1!B"\npShOY:U3EU49HK*7Db[Jsdbo8\n2s\nvo0 M Lt6\ufeffwrId\'\ufeff3ZD\'laNq[s&uyqsa4E*L5Y;XM\ngb.l-\ufeffv)5Is0_RffH:4g6e1O L_RAxth-CsU2.h b5Jw,gt&LuF9Z-pirIZr84OX(.4Uhk[wzTK6zTKzmWz;!["S3ZJo5Bffo5Jpc\nL_DK6B!1Wwz;km[lwB9;qOzilhmC[;fDG)faB\'\'YS]nfcaX5G)8:hk[s9vzD.u[QDe1WGIp*onr9Z:Jvz;\'\ufeffe1F"FMH\'n.;s]n lrQ_M0\'Nt\n37P!pi*GrB\n hH3Vj71v\''

In [31]:
#############

On va créer des blocs de taille `block_size` avec le jeu d'entrainement et la prédiction sera un bloc de même taille décalé d'un token.

In [16]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("Quand l'entrée est", context, "l'objectif est", target)

Quand l'entrée est tensor([80]) l'objectif est tensor(1)
Quand l'entrée est tensor([80,  1]) l'objectif est tensor(1)
Quand l'entrée est tensor([80,  1,  1]) l'objectif est tensor(28)
Quand l'entrée est tensor([80,  1,  1, 28]) l'objectif est tensor(39)
Quand l'entrée est tensor([80,  1,  1, 28, 39]) l'objectif est tensor(42)
Quand l'entrée est tensor([80,  1,  1, 28, 39, 42]) l'objectif est tensor(39)
Quand l'entrée est tensor([80,  1,  1, 28, 39, 42, 39]) l'objectif est tensor(44)
Quand l'entrée est tensor([80,  1,  1, 28, 39, 42, 39, 44]) l'objectif est tensor(32)


In [17]:
###############